In [1]:
import traffic
import pandas as pd
from biblio_herramienta import herramienta

In [2]:
carpetaDatos = r'datos_sectores/'
nombreDatos = "datos_bilbao.csv"

## Se cargan los datos previamente descargasdos del sector Bilbao

In [3]:
vuelos = herramienta.cargardatos(carpetaDatos,nombreDatos)

In [4]:
vuelos.data.head()

,Unnamed: 0,alert,altitude,callsign,geoaltitude,groundspeed,hour,icao24,last_position,latitude,longitude,onground,spi,squawk,timestamp,track,vertical_rate
0,0,False,36000.0,FPO1364,NaN,391.0,2019-02-05 02:00:00+00:00,396668,2019-02-05 02:10:19.800000+00:00,42.334791,-3.236176,False,False,5337.0,2019-02-05 02:10:20+00:00,11.654737,0.0
1,1,False,36000.0,FPO1364,NaN,391.0,2019-02-05 02:00:00+00:00,396668,2019-02-05 02:10:20.831000064+00:00,42.336467,-3.235665,False,False,5337.0,2019-02-05 02:10:21+00:00,11.654737,0.0
2,2,False,36000.0,FPO1364,NaN,391.0,2019-02-05 02:00:00+00:00,396668,2019-02-05 02:10:20.831000064+00:00,42.336467,-3.235665,False,False,5337.0,2019-02-05 02:10:22+00:00,11.654737,0.0
3,3,False,36000.0,FPO1364,NaN,391.0,2019-02-05 02:00:00+00:00,396668,2019-02-05 02:10:22.258999808+00:00,42.338376,-3.235154,False,False,5337.0,2019-02-05 02:10:23+00:00,11.654737,0.0
4,4,False,36000.0,FPO1364,NaN,391.0,2019-02-05 02:00:00+00:00,396668,2019-02-05 02:10:22.258999808+00:00,42.338376,-3.235154,False,False,5337.0,2019-02-05 02:10:24+00:00,11.654737,0.0


In [5]:
vuelos.data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111087 entries, 0 to 2111086
Data columns (total 17 columns):
 #   Column         Dtype  
---  ------         -----  
 0   Unnamed: 0     int64  
 1   alert          bool   
 2   altitude       float64
 3   callsign       object 
 4   geoaltitude    float64
 5   groundspeed    float64
 6   hour           object 
 7   icao24         object 
 8   last_position  object 
 9   latitude       float64
 10  longitude      float64
 11  onground       bool   
 12  spi            bool   
 13  squawk         float64
 14  timestamp      object 
 15  track          float64
 16  vertical_rate  float64
dtypes: bool(3), float64(8), int64(1), object(5)
memory usage: 231.5+ MB


In [6]:
vuelos_prueba = vuelos # generamos otros df de vuelos para poder editarlo a gusto

In [7]:
def niveldevuelo(altitud_pies):
    # esta funcion genera el nivel de vuelo
    try:
        fl = altitud_pies/100
        return round(fl)
    except:
        pass
"""
def quitarcolumnas(df):
    # esta funcion quita las columnas que no se van a usar
    df.data.drop(['alert', 'geoaltitude','hour','last_position','spi','squawk'], axis=1)
"""

"\ndef quitarcolumnas(df):\n    # esta funcion quita las columnas que no se van a usar\n    df.data.drop(['alert', 'geoaltitude','hour','last_position','spi','squawk'], axis=1)\n"

### Se añado el nivel de vuelo como columna al df

In [8]:
vuelos_prueba.data["FL"] = vuelos_prueba.data["altitude"].apply(niveldevuelo)

In [9]:
vuelos_prueba.data.head()

,Unnamed: 0,alert,altitude,callsign,geoaltitude,groundspeed,hour,icao24,last_position,latitude,longitude,onground,spi,squawk,timestamp,track,vertical_rate,FL
0,0,False,36000.0,FPO1364,NaN,391.0,2019-02-05 02:00:00+00:00,396668,2019-02-05 02:10:19.800000+00:00,42.334791,-3.236176,False,False,5337.0,2019-02-05 02:10:20+00:00,11.654737,0.0,360.0
1,1,False,36000.0,FPO1364,NaN,391.0,2019-02-05 02:00:00+00:00,396668,2019-02-05 02:10:20.831000064+00:00,42.336467,-3.235665,False,False,5337.0,2019-02-05 02:10:21+00:00,11.654737,0.0,360.0
2,2,False,36000.0,FPO1364,NaN,391.0,2019-02-05 02:00:00+00:00,396668,2019-02-05 02:10:20.831000064+00:00,42.336467,-3.235665,False,False,5337.0,2019-02-05 02:10:22+00:00,11.654737,0.0,360.0
3,3,False,36000.0,FPO1364,NaN,391.0,2019-02-05 02:00:00+00:00,396668,2019-02-05 02:10:22.258999808+00:00,42.338376,-3.235154,False,False,5337.0,2019-02-05 02:10:23+00:00,11.654737,0.0,360.0
4,4,False,36000.0,FPO1364,NaN,391.0,2019-02-05 02:00:00+00:00,396668,2019-02-05 02:10:22.258999808+00:00,42.338376,-3.235154,False,False,5337.0,2019-02-05 02:10:24+00:00,11.654737,0.0,360.0


In [10]:
vuelos_prueba.data.drop(['alert', 'geoaltitude','hour','last_position','spi','squawk'], axis=1)

,Unnamed: 0,altitude,callsign,groundspeed,icao24,latitude,longitude,onground,timestamp,track,vertical_rate,FL
0,0,36000.0,FPO1364,391.0,396668,42.334791,-3.236176,False,2019-02-05 02:10:20+00:00,11.654737,0.0,360.0
1,1,36000.0,FPO1364,391.0,396668,42.336467,-3.235665,False,2019-02-05 02:10:21+00:00,11.654737,0.0,360.0
2,2,36000.0,FPO1364,391.0,396668,42.336467,-3.235665,False,2019-02-05 02:10:22+00:00,11.654737,0.0,360.0
3,3,36000.0,FPO1364,391.0,396668,42.338376,-3.235154,False,2019-02-05 02:10:23+00:00,11.654737,0.0,360.0
4,4,36000.0,FPO1364,391.0,396668,42.338376,-3.235154,False,2019-02-05 02:10:24+00:00,11.654737,0.0,360.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2111082,1887,31000.0,SWR92,478.0,4b191c,42.337305,-3.717211,False,2019-02-07 23:27:40+00:00,246.817163,0.0,310.0
2111083,1888,31000.0,SWR92,478.0,4b191c,42.336467,-3.719958,False,2019-02-07 23:27:41+00:00,246.817163,0.0,310.0
2111084,1889,31000.0,SWR92,478.0,4b191c,42.335678,-3.722430,False,2019-02-07 23:27:42+00:00,246.817163,0.0,310.0
2111085,1890,31000.0,SWR92,478.0,4b191c,42.334465,-3.726218,False,2019-02-07 23:27:43+00:00,246.817163,0.0,310.0


# Como eliminar columnas bajo una condición especifica
Las condicones puesta en orden son:
1. Que no estén en tierra los aviones
2. Que no estén subiendo o bajando
3. Que estén dentro de la altura del sector


In [11]:
# 1
aviones_entierra =  vuelos_prueba.data[vuelos_prueba.data["onground"] == True].index
vuelos_prueba.data.drop(aviones_entierra,inplace = True)
#dfObj.drop(indexNames , inplace=True)

In [12]:
# 2
aviones_cambiandoFL = vuelos_prueba.data[vuelos_prueba.data["vertical_rate"]!= 0.0].index
vuelos_prueba.data.drop(aviones_cambiandoFL,inplace = True)

In [13]:
# 3
aviones_debajosector = vuelos_prueba.data[vuelos_prueba.data["FL"] <= 345].index
vuelos_prueba.data.drop(aviones_debajosector,inplace = True)

In [14]:
vuelos_prueba.data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1093092 entries, 0 to 2110882
Data columns (total 18 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   Unnamed: 0     1093092 non-null  int64  
 1   alert          1093092 non-null  bool   
 2   altitude       1092598 non-null  float64
 3   callsign       1086737 non-null  object 
 4   geoaltitude    1057913 non-null  float64
 5   groundspeed    1093092 non-null  float64
 6   hour           1093092 non-null  object 
 7   icao24         1093092 non-null  object 
 8   last_position  1093092 non-null  object 
 9   latitude       1093092 non-null  float64
 10  longitude      1093092 non-null  float64
 11  onground       1093092 non-null  bool   
 12  spi            1093092 non-null  bool   
 13  squawk         1092422 non-null  float64
 14  timestamp      1093092 non-null  object 
 15  track          1093092 non-null  float64
 16  vertical_rate  1093092 non-null  float64
 17  FL      

In [15]:
vuelos_prueba.data["callsign"].nunique()

1385

In [16]:
vuelos_prueba

,,count
icao24,callsign,
4ca60f,IBK5BV,3684
396668,FPO1364,3066
4cace5,IBK6054,2975
396668,FPO1363,2916
4cace5,IBK6055,2345
49514c,TAP434M,2178
344495,GES381L,2052
4cacc6,RYR3PE,2036
4853d1,TRA6845,1926


## Limpieza de vuelos no validos
Hay muchos datos que están incompletos o no son validos. OpenSky incorpora funciones para poder limpiar estos datos de manera sencilla

In [17]:
vuelos_prueba = vuelos_prueba.clean_invalid()
vuelos_prueba = vuelos_prueba.drop_duplicates()
vuelos_prueba

,,count
icao24,callsign,
4ca60f,IBK5BV,3684
396668,FPO1364,3066
4cace5,IBK6054,2975
396668,FPO1363,2916
4cace5,IBK6055,2345
49514c,TAP434M,2178
344495,GES381L,2052
4cacc6,RYR3PE,2036
4853d1,TRA6845,1926
